# **NUS DATHATON 2026**

***2 Sons 2 Daughters***

In [5]:
!pip install numpy
!pip install pandas

^C


In [13]:
import numpy as np
import pandas as pd


ModuleNotFoundError: No module named 'seaborn'

# read data

In [ ]:
from google.colab import files
uploaded = files.upload("champions_group_data.xlsx")
fname = next(iter(uploaded))   # gets the uploaded filename
df_raw = pd.read_excel(fname)
print("Loaded:", fname, "shape:", df.shape)



#clean data


In [ ]:
import re
import numpy as np
import pandas as pd

def _normalise_colname(s: str) -> str:
    s = s.strip().lower()
    s = re.sub(r"[%()\/]", " ", s)
    s = re.sub(r"[^a-z0-9]+", "_", s)
    s = re.sub(r"_+", "_", s).strip("_")
    return s

def _clean_string_series(x: pd.Series) -> pd.Series:
    x = x.astype("string")
    x = x.str.replace(r"\s+", " ", regex=True).str.strip()
    x = x.replace(
        {
            "": pd.NA,
            "na": pd.NA, "n/a": pd.NA, "none": pd.NA, "null": pd.NA,
            "unknown": pd.NA, "not available": pd.NA, "not applicable": pd.NA,
        }
    )
    return x

def _to_float(x: pd.Series) -> pd.Series:
    if x.dtype.kind in "if":
        return x.astype("float")
    x = _clean_string_series(x)
    # remove currency symbols and commas, keep digits, dot, minus
    x = x.str.replace(r"[$£€,\s]", "", regex=True)
    # handle parentheses negatives like (123)
    x = x.str.replace(r"^\((.+)\)$", r"-\1", regex=True)
    # keep only valid number-like
    x = x.where(x.str.match(r"^-?\d+(\.\d+)?$", na=False), pd.NA)
    return x.astype("float")

def _bucket_midpoint(val: str):
    """
    Handles:
      '1 to 10'
      '11 - 50'
      '1,001 to 5,000'
      '100000+' or '100,000+'
    Returns midpoint as float or NaN.
    """
    if val is None or (isinstance(val, float) and np.isnan(val)):
        return np.nan
    s = str(val).strip().lower()
    if s in {"", "na", "n/a", "none", "null", "unknown"}:
        return np.nan

    s = s.replace(",", "")
    s = re.sub(r"\s+", " ", s)

    # plus buckets
    m = re.match(r"^(\d+)\s*\+?$", s)
    if m:
        return float(m.group(1))

    # ranges with 'to' or '-'
    m = re.match(r"^(\d+)\s*(to|-)\s*(\d+)$", s)
    if m:
        a = float(m.group(1))
        b = float(m.group(3))
        return (a + b) / 2.0

    return np.nan

def clean_company_data(df_raw: pd.DataFrame) -> pd.DataFrame:
    df = df_raw.copy()

    # 1) normalise column names
    df.columns = [_normalise_colname(c) for c in df.columns]

    # 2) standardise obvious string columns
    for c in df.columns:
        if df[c].dtype == "object":
            df[c] = _clean_string_series(df[c])

    # 3) ensure key IDs are strings if present
    for c in ["company_id", "id", "duns", "sic_code", "naics_code", "nace_code", "isic_code"]:
        if c in df.columns:
            df[c] = _clean_string_series(df[c]).astype("string")

    # 4) numeric conversions where expected
    numeric_candidates = [
        "employees_total",
        "revenue_usd",
        "year_found",
        "corporate_family_members",
        "it_budget",
        "it_spend",
        "latitude",
        "longitude",
    ]
    for c in numeric_candidates:
        if c in df.columns:
            df[c] = _to_float(df[c])

    # 5) bucket to midpoint conversion for device footprint columns
    bucket_cols = [
        "no_of_pc",
        "no_of_desktops",
        "no_of_laptops",
        "no_of_routers",
        "no_of_servers",
        "no_of_storage_devices",
    ]
    for c in bucket_cols:
        if c in df.columns:
            df[c] = df[c].map(_bucket_midpoint).astype("float")

    # 6) basic sanity fixes
    if "year_found" in df.columns:
        df.loc[(df["year_found"] < 1700) | (df["year_found"] > 2030), "year_found"] = np.nan

    if "employees_total" in df.columns:
        df.loc[df["employees_total"] < 0, "employees_total"] = np.nan

    if "revenue_usd" in df.columns:
        df.loc[df["revenue_usd"] < 0, "revenue_usd"] = np.nan

    # 7) derived features that help segmentation
    if "revenue_usd" in df.columns and "employees_total" in df.columns:
        df["revenue_per_employee"] = df["revenue_usd"] / df["employees_total"]
        df.loc[~np.isfinite(df["revenue_per_employee"]), "revenue_per_employee"] = np.nan

    if "it_spend" in df.columns and "revenue_usd" in df.columns:
        df["it_spend_to_revenue"] = df["it_spend"] / df["revenue_usd"]
        df.loc[~np.isfinite(df["it_spend_to_revenue"]), "it_spend_to_revenue"] = np.nan

    # 8) drop columns that are basically empty (tune threshold if needed)
    missing_rate = df.isna().mean()
    mostly_empty = missing_rate[missing_rate >= 0.99].index.tolist()
    df = df.drop(columns=mostly_empty)

    # 9) de duplicate rows if an obvious unique key exists
    for key in ["company_id", "id", "duns", "company_name"]:
        if key in df.columns:
            df = df.drop_duplicates(subset=[key], keep="first")
            break

    return df


df_clean = clean_company_data(df_raw)
print(df_raw.shape, "->", df_clean.shape)
df_clean.head()


In [ ]:
# === Step 1: Build the clustering feature table (X) ===
# Assumes you already have: df_clean (your cleaned dataframe)

import numpy as np
import pandas as pd

df = df_clean.copy()

# --- Core columns for segmentation (v1) ---
industry_cols = ["sic_code", "sic_description", "8_digit_sic_code", "8_digit_sic_description"]

size_cols = ["employees_total", "revenue_usd", "revenue_per_employee"]

structure_cols = ["entity_type", "corporate_family_members", "is_headquarters", "is_domestic_ultimate"]

it_cols = [
    "it_budget", "it_spend",
    "no_of_pc", "no_of_desktops", "no_of_laptops",
    "no_of_routers", "no_of_servers", "no_of_storage_devices",
    "it_spend_to_revenue"
]

geo_cols = ["country", "region"]  # keep it coarse to avoid exploding categories

# Prefer 8-digit SIC if available; else regular SIC; else fall back to SIC description
# (this avoids redundant, high-cardinality columns)
preferred_industry = []
if "8_digit_sic_code" in df.columns:
    preferred_industry.append("8_digit_sic_code")
elif "sic_code" in df.columns:
    preferred_industry.append("sic_code")

# Add a description field if it exists (optional but useful)
if "8_digit_sic_description" in df.columns:
    preferred_industry.append("8_digit_sic_description")
elif "sic_description" in df.columns:
    preferred_industry.append("sic_description")

core_cols = preferred_industry + size_cols + structure_cols + it_cols + geo_cols
core_cols = [c for c in core_cols if c in df.columns]  # keep only existing

# --- Derived hierarchy flags (avoid clustering on raw company name strings) ---
# These are boolean features summarising parent/ultimate presence.
name_cols_for_flags = ["parent_company", "global_ultimate_company", "domestic_ultimate_company"]
for c in name_cols_for_flags:
    if c in df.columns:
        df[c] = df[c].astype("string").str.strip()

if "parent_company" in df.columns:
    df["has_parent_company"] = df["parent_company"].notna()
else:
    df["has_parent_company"] = False

if "global_ultimate_company" in df.columns:
    df["has_global_ultimate"] = df["global_ultimate_company"].notna()
else:
    df["has_global_ultimate"] = False

if "domestic_ultimate_company" in df.columns:
    df["has_domestic_ultimate_company"] = df["domestic_ultimate_company"].notna()
else:
    df["has_domestic_ultimate_company"] = False

derived_flag_cols = ["has_parent_company", "has_global_ultimate", "has_domestic_ultimate_company"]

# --- Build X ---
X = df[core_cols + derived_flag_cols].copy()

# --- Quick sanity check summary ---
print("X shape:", X.shape)
print("\nColumns used:")
print(X.columns.tolist())

missing_pct = (X.isna().mean().sort_values(ascending=False) * 100).round(1)
print("\nTop 15 columns by missing %:")
print(missing_pct.head(15))

# Optional: view a few rows
X.head(5)


In [ ]:
import numpy as np
import pandas as pd

X2 = X.copy()

# Ensure numeric columns are truly numeric (float) and replace pd.NA with np.nan
for col in num_cols:
    # pd.to_numeric handles various forms of missing data (like pd.NA, or strings like 'N/A')
    # and converts them to np.nan upon conversion to float. 'coerce' turns unconvertible values to NaN.
    X2[col] = pd.to_numeric(X2[col], errors='coerce')

# Ensure categorical columns are 'object' dtype and replace any pd.NA with np.nan
for col in cat_cols:
    # Convert pandas 'string' dtype to Python 'object' dtype if present
    if pd.api.types.is_string_dtype(X2[col]) or X2[col].dtype.name == "string":
        X2[col] = X2[col].astype("object")
    # Replace any pd.NA that might still exist in object columns with np.nan
    X2[col] = X2[col].replace({pd.NA: np.nan})

# A final global replacement for any pd.NA that might have been missed, just to be safe.
# This ensures that before passing to sklearn, no pd.NA values remain anywhere.
X2 = X2.replace({pd.NA: np.nan})

X2.head()

In [ ]:
# Identify numeric vs categorical columns inside X
num_cols = X.select_dtypes(include=["number", "bool"]).columns.tolist()
cat_cols = [c for c in X.columns if c not in num_cols]

print("Numeric cols:", num_cols)
print("Categorical cols:", cat_cols)


In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.impute import SimpleImputer

numeric_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="median")),
    ("scaler", StandardScaler()),
])

categorical_pipe = Pipeline([
    ("imputer", SimpleImputer(strategy="most_frequent")),
    ("onehot", OneHotEncoder(handle_unknown="ignore")),
])

preprocess = ColumnTransformer([
    ("num", numeric_pipe, num_cols),
    ("cat", categorical_pipe, cat_cols),
])

X_mat = preprocess.fit_transform(X2)
print("Transformed shape:", X_mat.shape)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

ks = range(3, 11)
scores = []

for k in ks:
    km = KMeans(n_clusters=k, random_state=42, n_init="auto")
    labels = km.fit_predict(X_mat)
    s = silhouette_score(X_mat, labels)
    scores.append(s)
    print(f"k={k}  silhouette={s:.4f}")

best_k = list(ks)[int(max(range(len(scores)), key=lambda i: scores[i]))]
print("Best k:", best_k)


In [ ]:
kmeans = KMeans(n_clusters=best_k, random_state=42, n_init="auto")
segments = kmeans.fit_predict(X_mat)

df_segmented = df.copy()
df_segmented["segment"] = segments

df_segmented["segment"].value_counts().sort_index()


#new clustering

In [ ]:
# === PLAN A: Rule-based segmentation (Industry + Size + Structure + IT + Geo) ===
# Assumes you already have df_clean

import numpy as np
import pandas as pd
import re

df = df_clean.copy()

# ---------- helpers ----------
def _digits_only(x):
    if pd.isna(x):
        return np.nan
    s = str(x).strip()
    s = re.sub(r"\D+", "", s)  # keep digits only
    return s if s != "" else np.nan

def sic_prefix(x, n=2):
    s = _digits_only(x)
    if pd.isna(s):
        return np.nan
    if len(s) >= n:
        return s[:n]
    return s.zfill(n)

def safe_qcut(series, q=4, labels=None):
    """Quantile binning that won't crash if not enough unique values."""
    s = pd.to_numeric(series, errors="coerce")
    if s.notna().sum() < q * 5:
        # too few points -> fallback to 3 bins
        q = 3
        labels = labels[:3] if labels is not None else None
    try:
        return pd.qcut(s, q=q, labels=labels, duplicates="drop")
    except Exception:
        # fallback: all Unknown
        return pd.Series(pd.NA, index=series.index, dtype="string")

# ---------- 1) Industry bucket ----------
sic_col = "8_digit_sic_code" if "8_digit_sic_code" in df.columns else "sic_code"
df["sic_2digit"] = df[sic_col].map(lambda x: sic_prefix(x, n=2))

# ---------- 2) Size tiers (employees + revenue) ----------
for c in ["employees_total", "revenue_usd", "it_spend", "it_budget"]:
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce")

df["log_employees"] = np.log1p(df["employees_total"])
df["log_revenue"] = np.log1p(df["revenue_usd"])

df["size_emp_tier"] = safe_qcut(
    df["log_employees"], q=4, labels=["emp_s", "emp_m", "emp_l", "emp_xl"]
).astype("string")

df["size_rev_tier"] = safe_qcut(
    df["log_revenue"], q=4, labels=["rev_s", "rev_m", "rev_l", "rev_xl"]
).astype("string")

# ---------- 3) Corporate structure tier ----------
# Make sure boolean-like columns behave
for b in ["is_headquarters", "is_domestic_ultimate"]:
    if b in df.columns:
        # convert common string booleans to True/False
        if df[b].dtype.name in ["string", "object"]:
            df[b] = df[b].astype("string").str.lower().map({"true": True, "false": False})
        df[b] = df[b].fillna(False).astype(bool)
    else:
        df[b] = False

# derived presence flags
if "parent_company" in df.columns:
    df["has_parent_company"] = df["parent_company"].notna()
else:
    df["has_parent_company"] = False

if "global_ultimate_company" in df.columns:
    df["has_global_ultimate"] = df["global_ultimate_company"].notna()
else:
    df["has_global_ultimate"] = False

if "domestic_ultimate_company" in df.columns:
    df["has_domestic_ultimate_company"] = df["domestic_ultimate_company"].notna()
else:
    df["has_domestic_ultimate_company"] = False

def structure_tier(row):
    if row.get("is_headquarters", False):
        return "hq"
    if row.get("is_domestic_ultimate", False):
        return "domestic_ultimate"
    # If entity_type exists, use it as first signal
    et = str(row.get("entity_type", "")).lower()
    if "subsidi" in et:
        return "subsidiary"
    if "branch" in et:
        return "branch"
    if row.get("has_parent_company", False):
        return "subsidiary_like"
    if row.get("has_global_ultimate", False) or row.get("has_domestic_ultimate_company", False):
        return "member_of_group"
    return "standalone_like"

df["structure_tier"] = df.apply(structure_tier, axis=1).astype("string")

# ---------- 4) IT footprint tiers (spend + device footprint) ----------
if "it_spend" in df.columns:
    df["log_it_spend"] = np.log1p(df["it_spend"])
    df["it_spend_tier"] = safe_qcut(
        df["log_it_spend"], q=4, labels=["it_low", "it_mid", "it_high", "it_top"]
    ).astype("string")
else:
    df["it_spend_tier"] = pd.Series(pd.NA, index=df.index, dtype="string")

device_cols = [c for c in [
    "no_of_pc", "no_of_desktops", "no_of_laptops", "no_of_routers", "no_of_servers", "no_of_storage_devices"
] if c in df.columns]

df["device_total"] = df[device_cols].sum(axis=1, min_count=1)
df["log_device_total"] = np.log1p(df["device_total"])
df["device_tier"] = safe_qcut(
    df["log_device_total"], q=4, labels=["dev_low", "dev_mid", "dev_high", "dev_top"]
).astype("string")

# ---------- 5) Geography tiers ----------
# Keep it coarse. Use region if present, else country.
if "region" in df.columns:
    df["geo_tier"] = df["region"].astype("string")
else:
    df["geo_tier"] = df["country"].astype("string")

# ---------- 6) Build final segment label + id ----------
seg_parts = ["sic_2digit", "size_emp_tier", "size_rev_tier", "structure_tier", "it_spend_tier", "device_tier", "geo_tier"]
for c in seg_parts:
    df[c] = df[c].fillna("Unknown").astype("string")

df["segment_label"] = df[seg_parts].agg("|".join, axis=1)

# numeric id for convenience
seg_order = df["segment_label"].value_counts().index.tolist()
seg_map = {lab: i for i, lab in enumerate(seg_order)}
df["segment_id"] = df["segment_label"].map(seg_map).astype(int)

# ---------- 7) Quick sanity outputs ----------
counts = df["segment_id"].value_counts()
print("Rows:", df.shape[0])
print("Num segments:", counts.shape[0])
print("Top 10 segment sizes:\n", counts.head(10))
print("Segments with <10 companies:", int((counts < 10).sum()))

df[["segment_id", "segment_label"]].head(10)


# Get Secret Key

In [1]:
from google.colab import userdata
userdata.get('HF_TOKEN')

ModuleNotFoundError: No module named 'google.colab'

In [8]:
!pip install streamlit

   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
   ---------------------------------------- 0.0/12.3 MB ? eta -:--:--
    --------------------------------------- 0.3/12.3 MB ? eta -:--:--
   -- ------------------------------------- 0.8/12.3 MB 1.9 MB/s eta 0:00:07
   --- ------------------------------------ 1.0/12.3 MB 1.9 MB/s eta 0:00:07
   ----- ---------------------------------- 1.6/12.3 MB 1.8 MB/s eta 0:00:07
   ----- ---------------------------------- 1.8/12.3 MB 1.7 MB/s eta 0:00:07
   ------- -------------------------------- 2.4/12.3 MB 1.8 MB/s eta 0:00:06
   -------- ------------------------------- 2.6/12.3 MB 1.8 MB/s eta 0:00:06
   --------- ------------------------------ 2.9/12.3 MB 1.7 MB/s eta 0:00:06
   ---------- ----------------------------- 3.1/12.3 MB 1.7 MB/s eta 0:00:06
   ----------- ---------------------------- 3.7/12.3 MB 1.7 MB/s eta 0:00:06
   ------------- ----------

In [33]:
!pip install openpyxl

  Using cached openpyxl-3.1.5-py2.py3-none-any.whl.metadata (2.5 kB)
  Using cached et_xmlfile-2.0.0-py3-none-any.whl.metadata (2.7 kB)
Using cached openpyxl-3.1.5-py2.py3-none-any.whl (250 kB)
Using cached et_xmlfile-2.0.0-py3-none-any.whl (18 kB)

   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   -------------------- ------------------- 1/2 [openpyxl]
   ---------------------------------------- 2/2 [openpyxl]



In [19]:
%%writefile app.py

import streamlit as st

st.title("My First Streamlit App")
st.write("Hello, this is running from a Jupyter Notebook!")

user_input = st.text_input("Enter your name:", "Streamlit User")
st.write(f"Welcome, {user_input}!")

Overwriting app.py


In [20]:
import os

# Define the path to the streamlit configuration directory
streamlit_config_dir = os.path.expanduser("~/.streamlit")
os.makedirs(streamlit_config_dir, exist_ok=True)

# Create the credentials.toml file with a blank email
config_path = os.path.join(streamlit_config_dir, "credentials.toml")
with open(config_path, "w") as f:
    f.write('[general]\nemail = ""')

print(f"Configuration file created at: {config_path}")

Configuration file created at: C:\Users\celes/.streamlit\credentials.toml


In [21]:
!streamlit run app.py

^C


In [23]:
%%writefile app.py
import streamlit as st
st.title("Hello Streamlit")
st.sidebar.write("Sidebar works")
import subprocess, sys
p = subprocess.Popen([sys.executable, "-m", "streamlit", "run", "app.py", "--server.port", "8501"])
print("Started Streamlit PID:", p.pid)


Overwriting app.py


In [34]:
%%writefile app.py
import streamlit as st
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import os

st.set_page_config(page_title="Company Intelligence Explorer", layout="wide")
st.title("Company Segmentation & Intelligence Explorer")

# ---------- Load (upload to avoid path issues) ----------
uploaded = st.sidebar.file_uploader("Upload Excel (.xlsx)", type=["xlsx"])
if uploaded is None:
    st.info("Upload the dataset to begin.")
    st.stop()

@st.cache_data
def load_data(file) -> pd.DataFrame:
    df = pd.read_excel(file)
    df.columns = [c.strip() for c in df.columns]
    # standardize to snake_case-ish for your rule code
    df.columns = [c.lower().strip().replace(" ", "_").replace(".", "") for c in df.columns]
    return df

df = load_data(uploaded)

# ---------- Helpers ----------
def pick_col(df, candidates):
    lower = {c.lower(): c for c in df.columns}
    for cand in candidates:
        cand2 = cand.lower().strip().replace(" ", "_").replace(".", "")
        if cand2 in lower:
            return lower[cand2]
    return None

def safe_num(s):
    return pd.to_numeric(s, errors="coerce")

@st.cache_data
def add_rule_segments(df_clean: pd.DataFrame) -> pd.DataFrame:
    df = df_clean.copy()

    def _digits_only(x):
        if pd.isna(x):
            return np.nan
        s = str(x).strip()
        s = re.sub(r"\D+", "", s)
        return s if s != "" else np.nan

    def sic_prefix(x, n=2):
        s = _digits_only(x)
        if pd.isna(s):
            return np.nan
        return s[:n] if len(s) >= n else s.zfill(n)

    def safe_qcut(series, q=4, labels=None):
        s = pd.to_numeric(series, errors="coerce")
        if s.notna().sum() < q * 5:
            q = 3
            labels = labels[:3] if labels is not None else None
        try:
            return pd.qcut(s, q=q, labels=labels, duplicates="drop")
        except Exception:
            return pd.Series(pd.NA, index=series.index, dtype="string")

    # ---------- 1) Industry bucket ----------
    sic_col = "8_digit_sic_code" if "8_digit_sic_code" in df.columns else ("sic_code" if "sic_code" in df.columns else None)
    df["sic_2digit"] = df[sic_col].map(lambda x: sic_prefix(x, n=2)) if sic_col else "Unknown"

    # ---------- 2) Size tiers ----------
    for c in ["employees_total", "revenue_usd", "it_spend", "it_budget"]:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")

    if "employees_total" in df.columns:
        df["log_employees"] = np.log1p(df["employees_total"])
        df["size_emp_tier"] = safe_qcut(df["log_employees"], q=4, labels=["emp_s", "emp_m", "emp_l", "emp_xl"]).astype("string")
    else:
        df["size_emp_tier"] = "Unknown"

    if "revenue_usd" in df.columns:
        df["log_revenue"] = np.log1p(df["revenue_usd"])
        df["size_rev_tier"] = safe_qcut(df["log_revenue"], q=4, labels=["rev_s", "rev_m", "rev_l", "rev_xl"]).astype("string")
    else:
        df["size_rev_tier"] = "Unknown"

    # ---------- 3) Structure tier ----------
    for b in ["is_headquarters", "is_domestic_ultimate"]:
        if b in df.columns:
            if df[b].dtype.name in ["string", "object"]:
                df[b] = df[b].astype("string").str.lower().map({"true": True, "false": False})
            df[b] = df[b].fillna(False).astype(bool)
        else:
            df[b] = False

    df["has_parent_company"] = df["parent_company"].notna() if "parent_company" in df.columns else False
    df["has_global_ultimate"] = df["global_ultimate_company"].notna() if "global_ultimate_company" in df.columns else False
    df["has_domestic_ultimate_company"] = df["domestic_ultimate_company"].notna() if "domestic_ultimate_company" in df.columns else False

    def structure_tier(row):
        if row.get("is_headquarters", False):
            return "hq"
        if row.get("is_domestic_ultimate", False):
            return "domestic_ultimate"
        et = str(row.get("entity_type", "")).lower()
        if "subsidi" in et:
            return "subsidiary"
        if "branch" in et:
            return "branch"
        if row.get("has_parent_company", False):
            return "subsidiary_like"
        if row.get("has_global_ultimate", False) or row.get("has_domestic_ultimate_company", False):
            return "member_of_group"
        return "standalone_like"

    df["structure_tier"] = df.apply(structure_tier, axis=1).astype("string")

    # ---------- 4) IT tiers + device tiers ----------
    if "it_spend" in df.columns:
        df["log_it_spend"] = np.log1p(df["it_spend"])
        df["it_spend_tier"] = safe_qcut(df["log_it_spend"], q=4, labels=["it_low", "it_mid", "it_high", "it_top"]).astype("string")
    else:
        df["it_spend_tier"] = "Unknown"

    device_cols = [c for c in ["no_of_pc", "no_of_desktops", "no_of_laptops", "no_of_routers", "no_of_servers", "no_of_storage_devices"] if c in df.columns]
    df["device_total"] = df[device_cols].sum(axis=1, min_count=1) if device_cols else np.nan
    df["log_device_total"] = np.log1p(df["device_total"])
    df["device_tier"] = safe_qcut(df["log_device_total"], q=4, labels=["dev_low", "dev_mid", "dev_high", "dev_top"]).astype("string")

    # ---------- 5) Geo tiers ----------
    if "region" in df.columns:
        df["geo_tier"] = df["region"].astype("string")
    elif "country" in df.columns:
        df["geo_tier"] = df["country"].astype("string")
    else:
        df["geo_tier"] = "Unknown"

    # ---------- 6) Final label/id ----------
    seg_parts = ["sic_2digit", "size_emp_tier", "size_rev_tier", "structure_tier", "it_spend_tier", "device_tier", "geo_tier"]
    for c in seg_parts:
        df[c] = df[c].fillna("Unknown").astype("string")

    df["segment_label"] = df[seg_parts].agg("|".join, axis=1)
    seg_order = df["segment_label"].value_counts().index.tolist()
    seg_map = {lab: i for i, lab in enumerate(seg_order)}
    df["segment_id"] = df["segment_label"].map(seg_map).astype(int)

    return df

df = add_rule_segments(df)

# ---------- Sidebar filters ----------
st.sidebar.subheader("Filters")
col_country = pick_col(df, ["country"])
col_entity  = pick_col(df, ["entity_type"])
col_name    = pick_col(df, ["company_name", "name", "company"])

def multiselect_filter(label, col):
    if col is None:
        st.sidebar.caption(f"⚠️ {label}: column not found")
        return []
    vals = sorted([v for v in df[col].dropna().astype(str).unique()])
    return st.sidebar.multiselect(label, vals)

sel_country = multiselect_filter("Country", col_country)
sel_entity  = multiselect_filter("Entity Type", col_entity)

seg_vals = sorted(df["segment_label"].dropna().astype(str).unique())
sel_segs = st.sidebar.multiselect("Segment", seg_vals)

filtered = df.copy()
if col_country and sel_country:
    filtered = filtered[filtered[col_country].astype(str).isin(sel_country)]
if col_entity and sel_entity:
    filtered = filtered[filtered[col_entity].astype(str).isin(sel_entity)]
if sel_segs:
    filtered = filtered[filtered["segment_label"].astype(str).isin(sel_segs)]

st.sidebar.caption(f"Filtered rows: {len(filtered):,}")

# ---------- Tabs ----------
tab1, tab2 = st.tabs(["Explore Companies", "Explore Segments"])

with tab1:
    st.subheader("Company Explorer")

    if col_name:
        company = st.selectbox("Select a company", sorted(filtered[col_name].fillna("UNKNOWN").astype(str).unique()))
        row = filtered[filtered[col_name].astype(str) == str(company)].head(1)
        st.write("Company record")
        st.dataframe(row, use_container_width=True)

    st.subheader("Filtered preview (top 200 rows)")
    st.dataframe(filtered.head(200), use_container_width=True)

with tab2:
    st.subheader("Segment Summary")
    seg_counts = filtered["segment_label"].value_counts().reset_index()
    seg_counts.columns = ["segment_label", "count"]
    st.dataframe(seg_counts.head(30), use_container_width=True)

    st.subheader("Top segments chart")
    top = seg_counts.head(15)
    fig = plt.figure()
    plt.bar(top["segment_label"].astype(str), top["count"])
    plt.xticks(rotation=90)
    st.pyplot(fig, clear_figure=True)


Overwriting app.py


In [35]:
import subprocess, sys
p = subprocess.Popen([sys.executable, "-m", "streamlit", "run", "app.py", "--server.port", "8502", "--server.headless", "true"])
print("Open: http://localhost:8502  |  PID:", p.pid)



Open: http://localhost:8502  |  PID: 27028


In [36]:
import socket, subprocess, sys, time, webbrowser

def get_free_port():
    s = socket.socket()
    s.bind(("", 0))
    port = s.getsockname()[1]
    s.close()
    return port

port = get_free_port()

p = subprocess.Popen(
    [sys.executable, "-m", "streamlit", "run", "app.py",
     "--server.port", str(port),
     "--server.headless", "true"],
    stdout=subprocess.PIPE,
    stderr=subprocess.STDOUT,
    text=True
)

time.sleep(1)
url = f"http://localhost:{port}"
print("OPEN THIS:", url, "| PID:", p.pid)

# try to open browser
webbrowser.open(url)

# print first ~30 log lines so you can see if it crashed
for _ in range(30):
    line = p.stdout.readline()
    if not line:
        break
    print(line, end="")


OPEN THIS: http://localhost:51555 | PID: 32092

  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:51555
  Network URL: http://10.249.51.38:51555
  External URL: http://137.132.26.219:51555

2026-01-19 13:28:31.551 Uncaught app execution
Traceback (most recent call last):
  File "C:\Users\celes\AppData\Local\Programs\Python\Python313\Lib\site-packages\streamlit\runtime\scriptrunner\exec_code.py", line 129, in exec_func_with_error_handling
    result = func()
  File "C:\Users\celes\AppData\Local\Programs\Python\Python313\Lib\site-packages\streamlit\runtime\scriptrunner\script_runner.py", line 672, in code_to_exec
    exec(code, module.__dict__)  # noqa: S102
    ~~~~^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\celes\Documents\GitHub\datathon-2s2d\app.py", line 150, in <module>
    df = add_rule_segments(df)
  File "C:\Users\celes\AppData\Local\Programs\Python\Python313\Lib\site-packages\streamlit\runtime\caching\cache_utils.py", line 281, in __call__
    r